In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys


In [3]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [ ]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json') #Update with your deck file

In [ ]:
syringe_hamilton = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json") # Update with your syringe file

In [ ]:
syringe_10cc_glass = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_glass_syringe.json") # Update with your syringe file

In [7]:
jubilee.load_tool(syringe_hamilton)
jubilee.load_tool(syringe_10cc_glass)

## Fill 1cc syringe

In [ ]:
syringe_hamilton.load_syringe(1, 1600) # dummy load

Loaded syringe, remaining volume 1 uL


In [ ]:
syringe_hamilton.set_pulsewidth(syringe_hamilton.empty_position-1, s = 2000) # set to empty position

Now place the syringe tip in a vial of water

In [9]:
syringe_hamilton.set_pulsewidth(syringe_hamilton.full_position+1, s = 10)

If purging, turn syringe upside down and cover tip

In [10]:
syringe_hamilton.set_pulsewidth(syringe_hamilton.full_position+400, s = 500)

In [11]:
syringe_hamilton.load_syringe(380, syringe_hamilton.full_position+400)

Loaded syringe, remaining volume 380 uL


## Fill 10cc syringe

In [12]:
syringe_10cc_glass.load_syringe(1, 1600)

Loaded syringe, remaining volume 1 uL


In [20]:
syringe_10cc_glass.set_pulsewidth(syringe_10cc_glass.empty_position-1, s = 2000)

In [13]:
syringe_10cc_glass.set_pulsewidth(syringe_10cc_glass.full_position+1, s = 10)

In [14]:
syringe_10cc_glass.set_pulsewidth(syringe_10cc_glass.full_position+250, s = 500)

In [15]:
syringe_10cc_glass.load_syringe(5900, syringe_10cc_glass.full_position+250)

Loaded syringe, remaining volume 5900 uL


## Load labware

In [16]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(17.8,  174.6), (131.2, 175.3), (131.6, 105.4)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [17]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(30.6, 264.2), (116.2, 264.8),  (117.0, 208.6)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [ ]:
# hack to work around labware with missing wells at B2, B7, E2, E7 positions

rows = ['A', 'B', 'C', 'D', 'E', 'F']
cols = [1, 2, 3, 4, 5, 6, 7, 8]

wells = []
for row in rows:
    for col in cols:
        if row in ['B', 'E'] and col in [2, 7]:
            pass
        else:
            wells.append(row + str(col))

In [ ]:
#check alignment with vials
for i, well in enumerate(wells):
    loc = samples[well]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = well
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue

174.6


Is the tip centered over sample A1? 


174.69635205793185


Is the tip centered over sample A2? 


174.7927041158637


...

Is the tip centered over sample B5? 


161.10177865861525


Is the tip centered over sample B6? 


161.29448277447898


KeyboardInterrupt: Interrupted by user

# Calibration

In [20]:
water_stocks = [stocks[0], stocks[1], stocks[2], stocks[3], stocks[4], stocks[5], stocks[6], stocks[7]]

In [ ]:
pulsewidths = [10, 50, 100] #suggested to use 50, 100, 250, 500

In [68]:
jubilee.safe_z_movement()
jubilee.pickup_tool(syringe_hamilton)

for i, pw in enumerate(pulsewidths):
    jubilee.move_to(z = jubilee.deck.safe_z+10)
    jubilee.move_to(x = water_stocks[i].x, y = water_stocks[i].y)
    jubilee.move_to(z = water_stocks[i].bottom(+5).point[2])
    syringe_hamilton.set_pulsewidth(syringe_hamilton.full_position+1)
    jubilee.safe_z_movement()
    current_pw = syringe_hamilton.full_position+1
    for j in range(5):
        well = wells[i*5+j]
        loc = samples[well]
        print(f'dispensing {pw} in well {well}')
        jubilee.safe_z_movement()
        jubilee.move_to(x = loc.x, y = loc.y)
        jubilee.move_to(z = loc.bottom(+5).point[2])
        syringe_hamilton.set_pulsewidth(current_pw + pw)
        current_pw += pw
        time.sleep(10)
        jubilee.safe_z_movement()

dispensing 10 in well A1
dispensing 10 in well A2
dispensing 10 in well A3
dispensing 10 in well A4
dispensing 10 in well A5
dispensing 50 in well A6
dispensing 50 in well A7
dispensing 50 in well A8
dispensing 50 in well B1
dispensing 50 in well B3
dispensing 100 in well B4
dispensing 100 in well B5
dispensing 100 in well B6
dispensing 100 in well B8
dispensing 100 in well C1


In [70]:
pw = 500
for j in range(5):
    #refill
    jubilee.move_to(z = jubilee.deck.safe_z+10)
    jubilee.move_to(x = water_stocks[3].x, y = water_stocks[i].y)
    jubilee.move_to(z = water_stocks[3].bottom(+5).point[2])
    syringe_hamilton.set_pulsewidth(syringe_hamilton.full_position+1)
    jubilee.safe_z_movement()
    current_pw = syringe_hamilton.full_position+1

    
    well = wells[15+j]
    loc = samples[well]
    print(f'dispensing {pw} in well {well}')
    jubilee.safe_z_movement()
    jubilee.move_to(x = loc.x, y = loc.y)
    jubilee.move_to(z = loc.bottom(+5).point[2])
    syringe_hamilton.set_pulsewidth(current_pw + pw)
    current_pw += pw
    time.sleep(10)
    jubilee.safe_z_movement()

dispensing 500 in well C2
dispensing 500 in well C3
dispensing 500 in well C4
dispensing 500 in well C5
dispensing 500 in well C6


In [69]:
wells[15]

'C2'

In [42]:
water_stocks[i].bottom(+5).point[2]

12.5

In [50]:
jubilee.move_to(x = water_stocks[i].x, y = water_stocks[i].y)

In [57]:
jubilee.move_to(z = water_stocks[i].bottom(+5).point[2])
syringe_hamilton.set_pulsewidth(syringe_hamilton.full_position+1)

In [58]:
jubilee.safe_z_movement()

In [56]:
syringe_hamilton.set_pulsewidth(syringe_hamilton.full_position + 500, s = 2000)

In [71]:
jubilee.park_tool()

## 10cc 

In [74]:
wells = wells[20:]

In [ ]:
pulsewidths = [10, 50, 100]

In [81]:
jubilee.safe_z_movement()
jubilee.pickup_tool(syringe_10cc_glass)

for i, pw in enumerate(pulsewidths):
    jubilee.move_to(z = jubilee.deck.safe_z+10)
    jubilee.move_to(x = water_stocks[i].x, y = water_stocks[i].y)
    jubilee.move_to(z = water_stocks[i].bottom(+5).point[2])
    syringe_10cc_glass.set_pulsewidth(syringe_10cc_glass.full_position+1)
    #backlash correction
    syringe_10cc_glass.set_pulsewidth(syringe_10cc_glass.full_position+15)
    jubilee.safe_z_movement()
    current_pw = syringe_10cc_glass.full_position+15
    for j in range(5):
        well = wells[i*5+j]
        loc = samples[well]
        print(f'dispensing {pw} in well {well}')
        jubilee.safe_z_movement()
        jubilee.move_to(x = loc.x, y = loc.y)
        jubilee.move_to(z = loc.bottom(+5).point[2])
        syringe_10cc_glass.set_pulsewidth(current_pw + pw)
        current_pw += pw
        time.sleep(10)
        jubilee.safe_z_movement()

dispensing 10 in well C7
dispensing 10 in well C8
dispensing 10 in well D1
dispensing 10 in well D2
dispensing 10 in well D3
dispensing 50 in well D4
dispensing 50 in well D5
dispensing 50 in well D6
dispensing 50 in well D7
dispensing 50 in well D8
dispensing 100 in well E1
dispensing 100 in well E3
dispensing 100 in well E4
dispensing 100 in well E5
dispensing 100 in well E6


In [82]:
jubilee.park_tool()

# Validation

In [33]:
################
## modify here ##
volumes = [50, 100, 250, 500, 1000]

###################
jubilee.pickup_tool(syringe_10cc_glass)
for i, vol in enumerate(volumes):
    syringe_10cc_glass.aspirate(syringe_10cc_glass.capacity - syringe_10cc_glass.remaining_volume - 1, water_stocks[i].bottom(+5))
    syringe_10cc_glass.dispense(500, stocks[11].bottom(+5), s = 10)
    for j in range(5):
        well = samples[wells[i*5+j]]
        print(f'dispensing {vol} into {well}')
        syringe_10cc_glass.dispense(vol, well.bottom(+20), s = 10)
        time.sleep(10)

jubilee.park_tool()

dispensing 50 into Well A1 form septavialrev1_44_holder_2000ul on slot 2
dispensing 50 into Well A2 form septavialrev1_44_holder_2000ul on slot 2
dispensing 50 into Well A3 form septavialrev1_44_holder_2000ul on slot 2
dispensing 50 into Well A4 form septavialrev1_44_holder_2000ul on slot 2
dispensing 50 into Well A5 form septavialrev1_44_holder_2000ul on slot 2
dispensing 100 into Well A6 form septavialrev1_44_holder_2000ul on slot 2
dispensing 100 into Well A7 form septavialrev1_44_holder_2000ul on slot 2
dispensing 100 into Well A8 form septavialrev1_44_holder_2000ul on slot 2
dispensing 100 into Well B1 form septavialrev1_44_holder_2000ul on slot 2
dispensing 100 into Well B3 form septavialrev1_44_holder_2000ul on slot 2
dispensing 250 into Well B4 form septavialrev1_44_holder_2000ul on slot 2
dispensing 250 into Well B5 form septavialrev1_44_holder_2000ul on slot 2
dispensing 250 into Well B6 form septavialrev1_44_holder_2000ul on slot 2
dispensing 250 into Well B8 form septavialr

In [27]:
stocks[11].bottom(+20)

Location(point=(116.79425573017066, 208.90668865837435, 27.5), labware=Well C4 form 20mLscintillation 12 Well Plate 18000 µL on slot 4)

In [31]:
jubilee.park_tool()